<h1><center>MERGE ACS DATA WITH FDNY DATA BY GEOID</center></h1>

In [25]:
import glob
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

years = [str(i) for i in range(2010, 2019)]
path_from = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\acs_data\\'
path_to = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\merged_acs_data\\'
base_path = 'C:/Users/mskac/machineLearning/GasLeakConEd/data/'

files = ['acs_merged_data' + str(year) + '.csv' for year in range(2010, 2019)]

In [26]:
dfs_acs_original = [pd.read_csv(join(path_to, files[i])) for i in range(9)]
for i in range(len(dfs_acs_original)):
    dfs_acs_original[i]['incident_year'] = 2010 +i

In [27]:
# we will have all years information in one dataframe to merge it with gas info
df_acs = pd.concat(dfs_acs_original, ignore_index = True)
df_acs['geoid'] = df_acs['geoid'].astype('int64')

In [28]:
df_gas = pd.read_csv(base_path+ 'processed\FULL_fdny_2013_2020.csv')
df_gas['geoid'] = df_gas['geoid'].astype('int64')

In [29]:
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['count'] = 1
df_gas_.head()

,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,count
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,1
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,1
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,1
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,1
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,1


In [30]:
df_gas_ = df_gas_.groupby(['incident_year', 'geoid']).agg({'engines_assigned_quantity': 'mean', 'total_incident_duration': 'mean', 'count': 'count'}).reset_index()

In [31]:
df_gas_.head()

,incident_year,geoid,engines_assigned_quantity,total_incident_duration,count
0,2013,36005000200,2.000000,1041.000000,2
1,2013,36005000400,2.200000,1351.000000,5
2,2013,36005001600,2.272727,1720.636364,11
3,2013,36005001900,2.000000,1170.000000,3
4,2013,36005002000,2.357143,1260.523810,42


In [32]:
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
df_acs['incident_year'] = df_acs['incident_year'].astype('int64')

In [33]:
merged_dfs = df_acs.merge(df_gas_, how='inner', on = ['incident_year','geoid'], validate="one_to_one")
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'geoid'])
merged_dfs.columns.to_list()

['Unnamed: 0',
 'avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'geoid',
 'unemployed%',
 'median_houshold_income',
 'mean_houshold_income',
 'total_housing_units',
 'occupied_housing_units%',
 'vacant_housing_units%',
 'total_population',
 'median_age',
 'not_us_citizen%',
 'incident_year',
 'engines_assigned_quantity',
 'total_incident_duration',
 'count']

In [34]:
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']

merged_dfs.rename(columns={'engines_assigned_quantity':'avg_assigned_fdny_engines#',
                          'total_incident_duration':'avg_incident_duration'}, inplace = True)
columns = ['incident_year','geoid','gas_leaks_per_person','avg_assigned_fdny_engines#',
 'avg_incident_duration',
 'median_age','avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'unemployed%',
 'median_houshold_income',
 'mean_houshold_income',
 'total_housing_units',
 'occupied_housing_units%',
 'vacant_housing_units%',
 'not_us_citizen%',
 ]

In [35]:
merged_dfs = merged_dfs[columns]
merged_dfs[merged_dfs < 0] = None
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11764 entries, 0 to 10181
Data columns (total 18 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   incident_year                                            11764 non-null  int64  
 1   geoid                                                    11764 non-null  int64  
 2   gas_leaks_per_person                                     11764 non-null  float64
 3   avg_assigned_fdny_engines#                               11764 non-null  float64
 4   avg_incident_duration                                    11764 non-null  float64
 5   median_age                                               11618 non-null  float64
 6   avg_houshold_size                                        11595 non-null  float64
 7   nonfamily_housholds%                                     11604 non-null  float64
 8   lonely_housholder%        

In [36]:
merged_dfs.to_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data.csv')

In [37]:
merged_dfs.head(5000)

,incident_year,geoid,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%
0,2013,36005000200,0.000412,2.000000,1041.000000,34.5,3.60,19.005197,18.040089,11.210097,9.576837,8.5,69514.0,72652.0,1476,91.3,8.7,11.198024
1,2013,36005000400,0.000936,2.200000,1351.000000,35.2,3.04,18.881917,13.519681,7.187678,4.164290,5.7,73036.0,80733.0,1912,91.7,8.3,7.821856
2,2013,36005001600,0.002109,2.272727,1720.636364,39.2,2.55,34.035990,33.470437,13.984576,1.748072,6.6,30817.0,44609.0,2043,95.2,4.8,15.819751
3,2013,36005001900,0.001171,2.000000,1170.000000,30.5,2.60,44.645006,35.499398,2.406739,0.000000,18.5,33862.0,42285.0,891,93.3,6.7,24.863388
4,2013,36005002000,0.004967,2.357143,1260.523810,35.9,2.84,29.798658,28.624161,12.953020,6.241611,13.5,19150.0,34044.0,3161,94.3,5.7,9.425260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5061,2015,36061015400,0.005241,1.974026,1542.935065,38.3,1.84,58.249749,49.435807,17.163992,0.000000,6.8,80803.0,101233.0,8418,94.7,5.3,12.842850
5062,2015,36061015500,0.000690,2.000000,1866.142857,43.8,1.86,57.724024,49.935862,21.770203,0.000000,7.5,106157.0,155370.0,6623,82.4,17.6,6.363905
5063,2015,36061015601,0.002034,2.454545,1510.181818,33.4,1.97,64.366557,44.103687,8.725812,2.628697,2.5,88620.0,118397.0,3053,89.7,10.3,16.087278
5064,2015,36061015602,0.009923,2.090909,1232.363636,37.0,2.21,51.473684,46.210526,16.842105,5.052632,10.1,50455.0,85864.0,1107,85.8,14.2,11.998196


In [57]:
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['incident_month'] = df_gas['incident_date_time'].str[:2]
df_gas_['count'] = 1
df_gas_.head()

,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,count,incident_month
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,1,01
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,1,01
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,1,07
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,1,02
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,1,12


In [58]:
df_gas_ = df_gas_.groupby(['incident_year', 'incident_month','geoid']).agg({'engines_assigned_quantity': 'mean', 'total_incident_duration': 'mean', 'count': 'count'}).reset_index()

In [59]:
df_gas_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79776 entries, 0 to 79775
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   incident_year              79776 non-null  object 
 1   incident_month             79776 non-null  object 
 2   geoid                      79776 non-null  int64  
 3   engines_assigned_quantity  79776 non-null  float64
 4   total_incident_duration    79776 non-null  float64
 5   count                      79776 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 3.7+ MB


In [60]:
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
df_acs['incident_year'] = df_acs['incident_year'].astype('int64')
df_gas_['incident_month'] = df_gas_['incident_month'].astype('int64')

In [61]:
merged_dfs = df_gas_.merge(df_acs, how='left', on = ['incident_year','geoid'])
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'incident_month','geoid'])
merged_dfs.columns.to_list()

['incident_year',
 'incident_month',
 'geoid',
 'engines_assigned_quantity',
 'total_incident_duration',
 'count',
 'Unnamed: 0',
 'avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'unemployed%',
 'median_houshold_income',
 'mean_houshold_income',
 'total_housing_units',
 'occupied_housing_units%',
 'vacant_housing_units%',
 'total_population',
 'median_age',
 'not_us_citizen%']

In [62]:
merged_dfs.head(50)

,incident_year,incident_month,geoid,engines_assigned_quantity,total_incident_duration,count,Unnamed: 0,avg_houshold_size,nonfamily_housholds%,lonely_housholder%,...,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,total_population,median_age,not_us_citizen%
0,2013,1,36005002000,2.333333,1162.833333,6,93.0,2.840000e+00,29.798658,28.624161,...,6.241611,13.5,19150.0,34044.0,3161.0,94.3,5.7,8456.0,35.9,9.425260
1,2013,1,36005002400,2.000000,911.000000,1,95.0,-6.666667e+08,NaN,NaN,...,NaN,-666666666.0,-666666666.0,-666666666.0,0.0,-666666666.0,-666666666.0,0.0,-666666666.0,NaN
2,2013,1,36005002500,2.000000,1062.000000,1,96.0,2.820000e+00,32.579934,27.177508,...,1.433297,12.1,14628.0,26129.0,2063.0,87.9,12.1,5108.0,27.0,25.078309
3,2013,1,36005002701,2.000000,822.500000,2,97.0,3.000000e+00,21.152030,15.391879,...,1.227573,18.5,26094.0,36719.0,1105.0,95.8,4.2,3172.0,32.7,17.559899
4,2013,1,36005002800,2.000000,6190.000000,1,99.0,2.480000e+00,39.102564,32.649573,...,0.000000,9.0,48717.0,58256.0,2427.0,96.4,3.6,5804.0,39.9,6.805651
5,2013,1,36005003500,2.500000,996.500000,2,102.0,2.910000e+00,30.256012,27.618309,...,0.000000,14.9,24321.0,30035.0,1346.0,95.8,4.2,3776.0,28.7,17.743644
6,2013,1,36005004001,2.000000,1323.000000,1,106.0,3.000000e+00,33.265306,30.000000,...,3.061224,9.8,46809.0,66040.0,564.0,86.9,13.1,1470.0,39.9,14.761905
7,2013,1,36005004100,2.000000,1203.500000,4,107.0,3.050000e+00,29.629630,29.234568,...,7.259259,16.3,18021.0,30864.0,2189.0,92.5,7.5,6218.0,25.9,9.295593
8,2013,1,36005004200,2.000000,1560.600000,5,108.0,2.750000e+00,34.641007,30.977054,...,3.515914,23.5,27868.0,37429.0,2797.0,96.6,3.4,7429.0,34.5,4.536277
9,2013,1,36005004300,2.000000,757.000000,1,109.0,2.930000e+00,33.941404,29.187396,...,3.758983,14.1,22359.0,35021.0,2005.0,90.2,9.8,5334.0,29.5,12.635921


In [63]:
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']

merged_dfs.rename(columns={'engines_assigned_quantity':'avg_assigned_fdny_engines#',
                          'total_incident_duration':'avg_incident_duration'}, inplace = True)
columns = ['incident_year','incident_month','geoid','gas_leaks_per_person']

In [64]:
merged_dfs = merged_dfs[columns]
merged_dfs[merged_dfs < 0] = None
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79776 entries, 0 to 79775
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   incident_year         79776 non-null  int64  
 1   incident_month        79776 non-null  int64  
 2   geoid                 79776 non-null  int64  
 3   gas_leaks_per_person  63785 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 3.0 MB


In [69]:
merged_dfs = merged_dfs.dropna()
merged_dfs.to_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data_months.csv')

In [70]:
merged_dfs.head(5000)

,incident_year,incident_month,geoid,gas_leaks_per_person
0,2013,1,36005002000,0.000710
1,2013,1,36005002400,inf
2,2013,1,36005002500,0.000196
3,2013,1,36005002701,0.000631
4,2013,1,36005002800,0.000172
...,...,...,...,...
5115,2013,8,36061002700,0.003359
5116,2013,8,36061002800,0.000274
5117,2013,8,36061003200,0.000393
5118,2013,8,36061003601,0.000909


In [71]:
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63785 entries, 0 to 65446
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   incident_year         63785 non-null  int64  
 1   incident_month        63785 non-null  int64  
 2   geoid                 63785 non-null  int64  
 3   gas_leaks_per_person  63785 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 2.4 MB


In [98]:
gas_df = pd.read_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data_months.csv')
centers_df = pd.read_csv(
    'https://raw.githubusercontent.com/MarinaOrzechowski/GasLeakConEd/timeline_branch/data/processed/important_(used_in_app)/geoid_with_centers.csv')

In [99]:
df_selected = gas_df.merge(centers_df, on='geoid')
df_selected = df_selected.groupby(['incident_year', 'incident_month', 'geoid']).agg(
        {'gas_leaks_per_person': 'mean'}).reset_index()
df_selected.head()

,incident_year,incident_month,geoid,gas_leaks_per_person
0,2013,1,36005002000,0.000710
1,2013,1,36005002400,inf
2,2013,1,36005002500,0.000196
3,2013,1,36005002701,0.000631
4,2013,1,36005002800,0.000172


In [100]:
df_selected = df_selected[df_selected['gas_leaks_per_person'] <1000000]

In [170]:
months = [month for month in range(1, 13)]
df_selected = df_selected.groupby(['incident_year','incident_month']).agg({'gas_leaks_per_person': 'mean'}).reset_index()

[df_selected[(df_selected['incident_year'] == year) & (
    df_selected['incident_month'] == month)].iloc[0]['gas_leaks_per_person'] for month in months]

[0.002257100623390955,
 0.0026059530957740595,
 0.0011248553907257557,
 0.0011507010118710272,
 0.0024853729568413835,
 0.003502722318995329,
 0.0029322065585796063,
 0.0016937637908488003,
 0.0021242811756353014,
 0.0039590416107056405,
 0.002018956409711787,
 0.0041142441686094645]

In [147]:
gas_leaks[2][2]

0.0011248553907257557